# Project: Improving Disaster Resilience

## NOAA Historical Events Files - Data Preparation and Consolidation

### 
### Disaster Resilience Project

### Data Used: NOAA Storm Events Details from 1996 to 2022

Andrew Sommers

### Purpose

Merge annual event files from NOAA into a countinuous dataset of NOAA Events.
Convert the Property Damage column to a numeric value.
Convert the Corp Damage column to a numeric value.
Create a Unique Event for NOAA events as events that cross U.S. States will have different event numbers for each state:  
    CUSTOM_EVENT_CODE provides a code for each unique NOAA event removing the duplication of the event across states. 
Output the combined NOAA event files. 


#### History 🗓️

Date | Person | Details
---- | ------ | -------
04/12/2023| Andrew Sommers|   Create initial notebook
06/01/2023| Andrew Sommers|   Adjust the working directory to use the NOAA folder under the Data folder
08/21/2023| Andrew Sommers|   Update the documentation for the notebook.



### Importing Libraries

Import required libraries

In [3]:
# import required libraries
#import requests # request http, api
import pandas as pd # tabluar data
#from functools import reduce
import numpy as np
import os


pd.options.display.max_columns = None # show all columns in display
pd.options.display.max_rows = None # show all rows in display

In [4]:
# change the notebooks data source directory to the Disaster Resilience  Directory - change this for your local environment
# set the sourcing directory where the 'Data' folder is located; this is the raw data input files.  
# this notebook assumes data files are located in a 'Data' folder in the following path:
os.chdir('C:\\Users\\andre\\OneDrive\\Documents\\IndianaUniversity\\D592\\Project_Disaster_Resilience\\Data\\NOAA')

In [5]:
# this function modifies the NOAA data to create values for DAMAGE_PROPERTY and DAMAGE_CROPS and pull a subset of columns
def get_NOAA_data(year):
    
    # read the input csv from the Data directory given the path in the prior cell - all columns set to type string
    NOAA_History_Adjusted = pd.read_csv(f'StormEvents_details-ftp_v1.0_d{year}.csv', dtype = str)
    
    #########################################################3
    # convert the property damage and crop damage fields to a numeric value.
    # The field is provided as a numeric value followed by a K, M, B, T as a multiplier of the value
    # This section of converts the string to a numeric value 
    # the following mapping is used to turn the Property Damage - last character - into the representative value
    value_replace_map = {
        'K': 1000,
        'M': 1000000,
        'B': 1000000000,
        'T': 1000000000000
    }
     
    # 
    # this section has to remove Nan and 'K' values to 0 from DAMAGE PROPERTY column
    # the  values (Example:  50.00K) must be converted to numeric values using the value_replace_table and lefthand characters of the column.
    NOAA_History_Adjusted['DAMAGE_PROPERTY'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'].fillna('0K') # replace  blanks with 0K
    NOAA_History_Adjusted['DAMAGE_PROPERTY'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'].replace(['0', 'K', 'M'],['0K', '0K', '0K']) # replace 0, K, and M with 0K
    NOAA_History_Adjusted['factor'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'].str[-1:] #get the last character from this column (will be K,M, B for thousands, millions, billions)
    NOAA_History_Adjusted['DAMAGE_PROPERTY'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'].str[:-1] #get all characters but the last character from the column - this is the actual numeric value
    NOAA_History_Adjusted['DAMAGE_PROPERTY'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'].astype(float) #change the string number value to a float value
    NOAA_History_Adjusted["factor"] = NOAA_History_Adjusted["factor"].map(lambda x: value_replace_map[x]) #use the value replace map to replace K,M, B with values
    NOAA_History_Adjusted["factor"] = NOAA_History_Adjusted["factor"].astype(float) #change the factor string number to a float value
    NOAA_History_Adjusted['DAMAGE_PROPERTY'] = NOAA_History_Adjusted['DAMAGE_PROPERTY'] * NOAA_History_Adjusted['factor'] #calculate the full value of the property damage
    NOAA_History_Adjusted = NOAA_History_Adjusted.drop(['factor'], axis = 1)   
     # this section has to remove Nan and 'K' values to 0 from DAMAGE PROPERTY column
    # the  values (Example:  50.00K) must be converted to numeric values using the value_replace_table and lefthand characters of the column.
    NOAA_History_Adjusted['DAMAGE_CROPS'] = NOAA_History_Adjusted['DAMAGE_CROPS'].fillna('0K') # replace  blanks with 0K
    NOAA_History_Adjusted['DAMAGE_CROPS'] = NOAA_History_Adjusted['DAMAGE_CROPS'].replace(['0', 'K', 'M'],['0K', '0K', '0K']) # replace 0, K, and M with 0K
    NOAA_History_Adjusted['factor'] = NOAA_History_Adjusted['DAMAGE_CROPS'].str[-1:] #get the last character from this column (will be K,M, B for thousands, millions, billions)
    NOAA_History_Adjusted['DAMAGE_CROPS'] = NOAA_History_Adjusted['DAMAGE_CROPS'].str[:-1] #get all characters but the last character from the column - this is the actual numeric value
    NOAA_History_Adjusted['DAMAGE_CROPS'] = NOAA_History_Adjusted['DAMAGE_CROPS'].astype(float) #change the string number value to a float value
    NOAA_History_Adjusted["factor"] = NOAA_History_Adjusted["factor"].map(lambda x: value_replace_map[x]) #use the value replace map to replace K,M, B with values
    NOAA_History_Adjusted["factor"] = NOAA_History_Adjusted["factor"].astype(float) #change the factor string number to a float value
    NOAA_History_Adjusted['DAMAGE_CROPS'] = NOAA_History_Adjusted['DAMAGE_CROPS'] * NOAA_History_Adjusted['factor'] #calculate the full value of the crop damage    
        
        
    #extract the required columns, rename the columns to align to the FEMA dataset
    NOAA_History_Analysis = NOAA_History_Adjusted[["YEAR", "EPISODE_ID", "EVENT_ID", "EVENT_TYPE", "EPISODE_NARRATIVE", "STATE", "STATE_FIPS", "CZ_NAME", "CZ_FIPS", "CZ_TYPE", "DAMAGE_PROPERTY", "DAMAGE_CROPS"]].copy()
    # Note:  The EPISODE_ID in the NOAA files does not provide a unique identifier of a NOAA event because
    # the same event can impact multiple states, and the EPISODE_ID is different for each state.
    # Using the Year, Month, Day, and Event Type in combination provides a good key to identify unique events that cross states
    NOAA_History_Analysis["CUSTOM_EVENT_CODE"] = NOAA_History_Adjusted["BEGIN_YEARMONTH"]+NOAA_History_Adjusted["BEGIN_DAY"]+NOAA_History_Adjusted["EPISODE_ID"] # create a unique key for NOAA events

     
    return NOAA_History_Analysis

In [6]:
# create a blank dataframe and concatenate each year of NOAA data into the dataframe
df_NOAA = pd.DataFrame(columns = ["YEAR", "EPISODE_ID", "EVENT_ID", "EVENT_TYPE", "EPISODE_NARRATIVE", "STATE", "STATE_FIPS", "CZ_NAME", "CZ_FIPS", "CZ_TYPE", "DAMAGE_PROPERTY", "DAMAGE_CROPS", "CUSTOM_EVENT_CODE"])

years = np.arange(1996,2023) 
 
for y in years:
    print("Getting data for year : ", y)
    df_NOAA = pd.concat([df_NOAA, get_NOAA_data(y)])
    
len(df_NOAA)

Getting data for year :  1996
Getting data for year :  1997
Getting data for year :  1998
Getting data for year :  1999
Getting data for year :  2000
Getting data for year :  2001
Getting data for year :  2002
Getting data for year :  2003
Getting data for year :  2004
Getting data for year :  2005
Getting data for year :  2006
Getting data for year :  2007
Getting data for year :  2008
Getting data for year :  2009
Getting data for year :  2010
Getting data for year :  2011
Getting data for year :  2012
Getting data for year :  2013
Getting data for year :  2014
Getting data for year :  2015
Getting data for year :  2016
Getting data for year :  2017
Getting data for year :  2018
Getting data for year :  2019
Getting data for year :  2020
Getting data for year :  2021
Getting data for year :  2022


1561601

In [7]:
df_NOAA.head(3)

,YEAR,EPISODE_ID,EVENT_ID,EVENT_TYPE,EPISODE_NARRATIVE,STATE,STATE_FIPS,CZ_NAME,CZ_FIPS,CZ_TYPE,DAMAGE_PROPERTY,DAMAGE_CROPS,CUSTOM_EVENT_CODE
0,1996,2052149,5570068,High Wind,Strong gradient winds followed behind a line o...,ILLINOIS,17,HANCOCK,34,Z,0.0,0.0,199610292052149
1,1996,2052748,5570362,Flash Flood,Flash flood waters washed out a small bridge a...,OKLAHOMA,40,BRYAN,13,C,0.0,0.0,199610212052748
2,1996,2052749,5570363,Hail,Hail as large as softballs broke out car and t...,OKLAHOMA,40,STEPHENS,137,C,0.0,0.0,199610202052749


In [8]:
# output the combined data to a csv file
os.chdir('C:\\Users\\andre\\OneDrive\\Documents\\IndianaUniversity\\D592\\Project_Disaster_Resilience\\Data\\NOAA')
df_NOAA.to_csv(f'NOAA_EventHistory_1996_2022.csv', header=True, index=False)
print('end of file combine and save function 1')

end of file combine and save function 1
